<br></br>
# **gensim | doc2vec**
<br></br>
## **1 네이버 단어 전처리**
네이버 영화리뷰 단어모델 만들기

In [1]:
from konlpy.tag import Okt
twitter = Okt()

def read_data(filename):
    with open(filename, 'r') as f:
        data = [line.split('\t') for line in f.read().splitlines()]
    from random import randint
    random_data = [data[randint(1, len(data))]  for no in range(int(len(data)/10)) ]
    return random_data

def tokenize(doc):
    return ['/'.join(t) for t in twitter.pos(doc, norm=True, stem=True)]

In [2]:
%%time
from collections import namedtuple
train_data        = read_data('data/ratings_train.txt')
train_docs        = [(tokenize(row[1]), row[2]) for row in train_data[1:]]
TaggedDocument    = namedtuple('TaggedDocument', 'words tags')
tagged_train_docs = [TaggedDocument(d, [c]) for d, c in train_docs]

CPU times: user 54.8 s, sys: 304 ms, total: 55.1 s
Wall time: 40.5 s


In [3]:
from pprint import pprint
pprint(tagged_train_docs[0])

TaggedDocument(words=['감동/Noun', '그/Noun', '자체/Noun', '.../Punctuation', '여자/Noun', '로서/Noun', '의/Josa', '품위/Noun', '를/Josa', '지키다/Verb', '위해/Noun', '선생님/Noun', '의/Josa', '품위/Noun', '를/Josa', '버리다/Verb', '.../Punctuation'], tags=['1'])


<br></br>
## **2 doc2vec 파라미터 설정 및 학습**

In [4]:
%%time
from gensim.models import doc2vec
doc_vectorizer = doc2vec.Doc2Vec(vector_size=300, alpha=0.025, min_alpha=0.025, seed=1234)
doc_vectorizer.build_vocab(tagged_train_docs)

for epoch in range(10):
    doc_vectorizer.train(tagged_train_docs, 
                         total_examples = doc_vectorizer.corpus_count, 
                         epochs = doc_vectorizer.epochs)
    doc_vectorizer.alpha -= 0.002
    doc_vectorizer.min_alpha = doc_vectorizer.alpha 

# 학습이 완료된 모델의 데이터를 저장한다
doc_vectorizer.save('data/doc2vec.model')

CPU times: user 56.6 s, sys: 3.81 s, total: 1min
Wall time: 29.7 s


<br></br>
## **3 doc2Vec 모델활용**

In [5]:
from gensim.models import doc2vec
from pprint import pprint
doc_vectorizer = doc2vec.Doc2Vec.load('data/doc2vec.model')

In [6]:
pprint(doc_vectorizer.wv.most_similar('공포/Noun'))

[('장르/Noun', 0.5214426517486572),
 ('스릴러/Noun', 0.46614718437194824),
 ('매니아/Noun', 0.41379544138908386),
 ('코미디/Noun', 0.4019935131072998),
 ('호러/Noun', 0.39797520637512207),
 ('범죄/Noun', 0.38795462250709534),
 ('토이스토리/Noun', 0.38510462641716003),
 ('분위기/Noun', 0.3645700514316559),
 ('흑역사/Noun', 0.36110109090805054),
 ('문화/Noun', 0.359845370054245)]


/home/markbaum/Python/nltk/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [7]:
doc_vectorizer.wv.similarity('공포/Noun', 'ㅋㅋ/KoreanParticle')

/home/markbaum/Python/nltk/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.12470643

In [8]:
pprint(doc_vectorizer.wv.most_similar(positive=['여자/Noun', '공포/Noun'], 
                                      negative=['남자/Noun']))

[('그야말로/Adverb', 0.3589343726634979),
 ('스릴러/Noun', 0.3531931936740875),
 ('어린이/Noun', 0.3464009463787079),
 ('장르/Noun', 0.33753374218940735),
 ('호러/Noun', 0.3230978846549988),
 ('그다지/Noun', 0.3225425183773041),
 ('재미나/Noun', 0.3217495083808899),
 ('신파/Noun', 0.3143833875656128),
 ('문화/Noun', 0.30652162432670593),
 ('무협/Noun', 0.30547401309013367)]


/home/markbaum/Python/nltk/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [11]:
doc_vectorizer.infer_vector(['픽사/Noun', '최고/Noun', '명작/Noun'])[:10]

array([ 0.00428942,  0.00562006, -0.00339408,  0.00772786,  0.00257091,
       -0.0075936 , -0.00884658, -0.00419246,  0.00110028,  0.00501144],
      dtype=float32)

In [16]:
doc_vectorizer.infer_vector(['픽사/Noun', '최고/Noun', '명작/Noun']).sum()

-0.1214823